In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.api as sm
import scipy
from scipy.stats import norm
from scipy.stats.mstats import gmean

from statsmodels.nonparametric.smoothers_lowess import lowess

from numpy.lib.stride_tricks import sliding_window_view

# Load data

In [3]:
df = pd.read_parquet("/Users/julianeoliveira/Downloads/aesop_2026_01_09_mun.parquet")

In [107]:
#df.columns.to_list()

In [3]:
lst = ['co_uf','nm_uf','nm_municipio','co_ibge', 'epiyear', 'epiweek','year_week', 'atend_totais', 'atend_ivas','ra_atend_ivas','ra_atend_ivas_ma',]

df = df[lst]

# Functions 

In [4]:
# Create season periods

def add_sea(data, year_col="epiyear", week_col="epiweek", n_week=42):
    """
    Add epidemiological season column based on a cutoff week.

    Season definition:
    - weeks >= n_week belong to the next year's season
    - weeks < n_week belong to the current year's season
    The season is labeled by the ending year.
    """

    col = f"season_w{n_week}"

    data[col] = data[year_col]
    data.loc[data[week_col] >= n_week, col] += 1

    return data


In [5]:
# --- helper: smooth MAP curve with LOWESS ---
def smooth_lowess(pr):
    r = np.arange(1, len(pr)+1)
    smoothed = lowess(pr, r, frac=0.3, return_sorted=False)
    return smoothed

In [6]:
def mem_epidemic_period(set_muni, col_year = 'epiyear', col_series = 'atend_ivas_ma',  delta=0.02):
    """
    Compute epidemic period (step 1 of MEM) for each season/year.

    Parameters
    ----------
    set_muni : DataFrame
        Must contain columns ['epiyear', 'atend_ivas'].
    delta : float, optional
        Threshold for slope increment (default=0.02).

    Returns
    -------
    summary : DataFrame
        Columns: ['epiyear', 'r_j_estr', 'k_start', 'k_end']
    details : dict
        Keys = epiyear, values = dict with:
            - r_j_estr : estimated epidemic duration (weeks)
            - k_start  : start week (1-indexed)
            - k_end    : end week (inclusive, 1-indexed)
            - p_j_r    : MAP percentages
            - sm_p_j_r : smoothed MAP curve
            - t_j_r    : cumulative totals by window length
    """
    
    results = {}
    summary_records = []

    for year in sorted(set_muni[col_year].unique()):
        s1 = set_muni[set_muni[col_year] == year].reset_index(drop=True)

        t = np.asarray(s1[col_series])
        S = len(t)
        tS = np.sum(t)

        # --- compute t_j_r (max accumulated per window length) ---
        t_j_r = np.array([
            sliding_window_view(t, r).sum(axis=1).max()
            for r in range(1, S + 1)
        ])

        # --- MAP curve ---
        p_j_r = t_j_r / tS
        sm_p_j_r = smooth_lowess(p_j_r)

        # --- increments Δr ---
        delta_j_r = np.diff(sm_p_j_r)

        # --- determine optimal duration r_j_estr ---
        indices = np.where(delta_j_r < delta)[0]
        r_j_estr = int(indices.min()) if len(indices) > 0 else S

        # --- find k_estr (start index of epidemic window) ---
        window_r = sliding_window_view(t, r_j_estr)
        window_sums = window_r.sum(axis=1)
        k_estr = window_sums.argmax()

        k_start = k_estr + 1          # 1-indexed
        k_end   = k_estr + r_j_estr   # inclusive

        # --- save results ---
        results[year] = {
            'r_j_estr': r_j_estr,
            'k_start': k_start,
            'k_end': k_end,
            'p_j_r': p_j_r,
            'sm_p_j_r': sm_p_j_r,
            't_j_r': t_j_r
        }

        summary_records.append({
            'epiyear': year,
            'r_j_estr': r_j_estr,
            'k_start': k_start,
            'k_end': k_end
        })

    summary = pd.DataFrame(summary_records)

    return summary, results


# Start run method

## Define some valid seasonalities

In [17]:
df = add_sea(df)
df = add_sea(df, n_week=40)

In [ ]:
#df = df[df.year_week >= '2021-42'].copy()

In [9]:
lst = []

for city in df.co_ibge.unique():
    
    set_muni = df[df.co_ibge == city].copy().reset_index()

    set_muni = set_muni.assign(atend_ivas_ma = set_muni["atend_ivas"].rolling(window=4, min_periods=1).mean()
                              )
    
    summary, details = mem_epidemic_period(set_muni, col_year = 'season_w42', col_series = 'atend_ivas_ma', delta=0.02)
    
    co_uf = set_muni.co_uf.unique(),
    nm_uf = set_muni.nm_uf.unique()
    
    summary = summary.assign(co_ibge = city,
                             co_uf = set_muni.co_uf.iloc[0],
                             nm_uf = set_muni.nm_uf.iloc[0]
                             )
    
    lst.append(summary)
    
    

/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_1790/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_1790/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129

In [10]:
final = pd.concat(lst)

# Select the cities that has a good season using season_w42

In [11]:
final.co_ibge.nunique()

5570

In [39]:
final_filtered = final[final["epiyear"].isin([2022, 2023, 2024]) & ( (final["k_start"] < 10) | (final["r_j_estr"] > 20) )]

In [40]:
final_filtered.co_ibge.nunique()

4137

In [29]:
set1_good = set(final.co_ibge.unique()) - set(final_filtered.co_ibge.unique())

In [76]:
set1_good

{172080,
 270390,
 311350,
 352320,
 172097,
 172100,
 352330,
 172120,
 172125,
 352400,
 352410,
 352420,
 270510,
 172208,
 172210,
 270540,
 352490,
 311550,
 352510,
 270600,
 311560,
 311570,
 270650,
 352580,
 311630,
 270680,
 311640,
 311670,
 500100,
 270740,
 352660,
 410010,
 311710,
 311730,
 410040,
 311740,
 410050,
 270800,
 270810,
 352730,
 311800,
 410105,
 320016,
 352800,
 270900,
 311870,
 270930,
 500320,
 320100,
 500330,
 352880,
 500345,
 230010,
 311930,
 352890,
 230020,
 410260,
 500380,
 230050,
 410280,
 230060,
 500400,
 320180,
 230075,
 312000,
 352960,
 320200,
 312010,
 410315,
 230100,
 230110,
 312030,
 410335,
 320225,
 140002,
 140005,
 320230,
 230120,
 410347,
 140015,
 140017,
 230130,
 410360,
 230140,
 320255,
 500500,
 410405,
 230190,
 500540,
 230205,
 320320,
 312140,
 353110,
 353120,
 410470,
 230250,
 312170,
 353140,
 230280,
 353160,
 320400,
 312210,
 500627,
 312230,
 353190,
 312235,
 230320,
 353210,
 410560,
 353220,
 320465,
 

In [56]:
final_filtered[final_filtered.r_j_estr >20]

,epiyear,r_j_estr,k_start,k_end,co_ibge,co_uf,nm_uf
2,2024,21,16,36,110002,11,Rondônia
1,2023,21,1,21,110003,11,Rondônia
0,2022,25,12,36,110004,11,Rondônia
1,2023,25,24,48,110004,11,Rondônia
2,2024,22,9,30,110012,11,Rondônia
...,...,...,...,...,...,...,...
2,2024,23,30,52,522180,52,Goiás
2,2024,22,27,48,522190,52,Goiás
2,2024,29,24,52,522200,52,Goiás
2,2024,29,20,48,522205,52,Goiás


# New tentative

In [ ]:
def summary_is_valid_old(summary, k_start_min=10, r_max=20):
    return (
        (summary["k_start"] > k_start_min).all()
        and
        (summary["r_j_estr"] <= r_max).all()
    )


In [18]:
def summary_is_valid(
    summary,
    years_to_check,
    k_start_min=10,
    r_max=20,
    year_col="epiyear",
):
    """
    Check validity conditions only for selected years.
    """

    sub = summary[summary[year_col].isin(years_to_check)]

    # if no rows for those years, treat as invalid (safer)
    if sub.empty:
        return False

    return (
        (sub["k_start"] > k_start_min).all()
        and
        (sub["r_j_estr"] <= r_max).all()
    )


In [19]:

def find_best_season_and_delta(
    df,
    city,
    season_weeks=(42, 32, 0),
    delta_start=0.02,
    delta_max=0.04,
    delta_step=0.001,
    k_start_min=10,
    r_max=20,
):
    # subset city
    set_muni = (
        df[df.co_ibge == city]
        .copy()
        .reset_index(drop=True)
    )

    # smooth series
    set_muni["atend_ivas_ma"] = (
        set_muni["atend_ivas"]
        .rolling(window=4, min_periods=1)
        .mean()
    )

    deltas = np.arange(delta_start, delta_max- 1e-9, +delta_step)

    for w in season_weeks:
        # create season definition
        set_muni = add_sea(set_muni, n_week=w)
        season_col = f"season_w{w}"

        for delta in deltas:
            
            summary, _ = mem_epidemic_period(
                set_muni,
                col_year=season_col,
                col_series="atend_ivas_ma",
                delta=delta,
            )


            if summary_is_valid(summary, years_to_check=[2023, 2024], k_start_min=k_start_min, r_max=r_max):
                out = summary.copy()
                out["co_ibge"] = city
                out["season_def"] = season_col
                out["delta_used"] = delta
                out["co_uf"] = set_muni.co_uf.iloc[0]
                out["nm_uf"] = set_muni.nm_uf.iloc[0]
                return out

    return None


### testt the code for one city

In [21]:
res = find_best_season_and_delta(
    df,
    110001,
    season_weeks=(42, 32, 0),
    delta_start=0.02,
    delta_max=0.04,
    delta_step=0.001,
    k_start_min=10,
    r_max=20,
)

In [22]:
print(res)


   epiyear  r_j_estr  k_start  k_end  co_ibge  season_def  delta_used  co_uf  \
0     2017        31        1     31   110001  season_w32        0.02     11   
1     2018        12       41     52   110001  season_w32        0.02     11   
2     2019        13       40     52   110001  season_w32        0.02     11   
3     2020         9       44     52   110001  season_w32        0.02     11   
4     2021        17       24     40   110001  season_w32        0.02     11   
5     2022        22        1     22   110001  season_w32        0.02     11   
6     2023         9       17     25   110001  season_w32        0.02     11   
7     2024        17       26     42   110001  season_w32        0.02     11   
8     2025        14       38     51   110001  season_w32        0.02     11   
9     2026         3       19     21   110001  season_w32        0.02     11   

      nm_uf  
0  Rondônia  
1  Rondônia  
2  Rondônia  
3  Rondônia  
4  Rondônia  
5  Rondônia  
6  Rondônia  
7  Rond

### run for all cities

In [ ]:
results = []

for city in df.co_ibge.unique():
    print(f"Processing {city}...")
    res = find_best_season_and_delta(df, city)
    if res is not None:
        results.append(res)

final_selected = pd.concat(results, ignore_index=True)


In [24]:
len(results)

5249

In [26]:
results[1]

,epiyear,r_j_estr,k_start,k_end,co_ibge,season_def,delta_used,co_uf,nm_uf
0,2017,10,7,16,110004,season_w42,0.023,11,Rondônia
1,2018,8,40,47,110004,season_w42,0.023,11,Rondônia
2,2019,11,18,28,110004,season_w42,0.023,11,Rondônia
3,2020,16,37,52,110004,season_w42,0.023,11,Rondônia
4,2021,11,17,27,110004,season_w42,0.023,11,Rondônia
5,2022,1,15,15,110004,season_w42,0.023,11,Rondônia
6,2023,20,24,43,110004,season_w42,0.023,11,Rondônia
7,2024,8,27,34,110004,season_w42,0.023,11,Rondônia
8,2025,16,24,39,110004,season_w42,0.023,11,Rondônia
9,2026,12,1,12,110004,season_w42,0.023,11,Rondônia


In [28]:

final_selected.to_parquet('/Users/julianeoliveira/Downloads/def_sea_MEM_23_01_2026.parquet')

In [ ]:
final_selected.r_j_estr.mean()

13.663370313695486

In [39]:
final_selected.season_def.value_counts()

season_def
season_w42    45130
season_w32     5260
season_w0      1890
Name: count, dtype: int64

In [30]:
final_selected.co_ibge.nunique()

5249

In [31]:
set_bad =  set(df.co_ibge.unique()) - set(final_selected.co_ibge.unique())

In [32]:
5570 - 5249

321

In [48]:

dta = pd.DataFrame({"co_ibge": sorted(set_bad)})
dta.to_csv("/Users/julianeoliveira/Downloads/set_bad_cities.csv", index=False)


In [36]:
30/14

2.142857142857143

# find weaker solution for the bad cities

In [40]:
len(results2 )

0

In [ ]:
results2 = []

for city in set_bad:
    print(f"Processing {city}...")
    res = find_best_season_and_delta(df, city, k_start_min=5, r_max=20)
    if res is not None:
        results.append(res)

final_selected2 = pd.concat(results2, ignore_index=True)

Processing 210945...
Processing 280580...
Processing 430085...
Processing 350250...


/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:1

Processing 352300...
Processing 411700...


/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:1

Processing 315445...
Processing 313420...
Processing 510030...
Processing 432210...
Processing 430163...
Processing 211027...
Processing 510040...
Processing 354400...


...

/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:1

Processing 270260...
Processing 352180...
Processing 260020...
Processing 210880...
Processing 522185...
Processing 520145...


/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/0r/rms2c9v53rj6wzhz7809886m0000gn/T/ipykernel_12740/2850365485.py:43: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/.pyenv/versions/3.11.8/lib/python3.11/site-packages/numpy/core/_methods.py:1

Processing 352210...
Processing 421850...
Processing 270300...
Processing 520160...
Processing 315370...
Processing 522220...
Processing 280560...
Processing 352240...
Processing 290800...
Processing 421880...


ValueError: No objects to concatenate